# Face Recognition #5

- We are using: DB1-Youtube (half database from A to I)
- We are performing new lighter embeddings for each person taking just random photograms
- We are then tuning tolerance and threshold parameter

In [48]:
import argparse
import pickle
import cv2
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
import dlib
import matplotlib.pyplot as plt
import numpy as np
import time

print(dlib.DLIB_USE_CUDA)
#dlib.DLIB_USE_CUDA = True

True


In [49]:
def createEncodingsLimited(dataset_folder, name, detection='cnn', enc_number = 20):
    dlib.DLIB_USE_CUDA = True

    # grab the paths to the input images in our dataset
    print("[INFO] creating encoding...")
    imagePaths = list(paths.list_images(dataset_folder))

    # initialize the list of known encodings and known names
    knownEncodings = []
    knownNames = []
    
    total_pictures = len(imagePaths)
    random_values = np.random.randint(0, total_pictures, size=enc_number)

    # loop over the image paths
    for i in random_values:
        imagePath = imagePaths[i]
        # extract the person name from the image path
        #print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]

        # load the input image and convert it from RGB (OpenCV ordering)
        # to dlib ordering (RGB)
        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb,model=detection)

        # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

        # loop over the encodings
        for encoding in encodings:
            # add each encoding + name to our set of known names and
            # encodings
            knownEncodings.append(encoding)
            knownNames.append(name)

    # dump the facial encodings + names to disk
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}
    name = "./encodings/" + name.split('/')[2] + '.pkl'
    print(name)
    f = open(name, "wb")
    f.write(pickle.dumps(data))
    f.close()

In [50]:
def faceRecognition(image, encodings, detection='cnn', show=False, tolerance = 0.6):
    start = time.time()
    # the image is already converted in an rgb format
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # format used by openCV
    data = encodings

    # detect the (x, y)-coordinates of the bounding boxes corresponding
    # to each face in the input image, then compute the facial embeddings
    # for each face
    boxes = face_recognition.face_locations(rgb,model=detection)
    encodings = face_recognition.face_encodings(rgb, boxes)

    # initialize the list of names for each face detected
    names = []

    # loop over the facial embeddings
    for encoding in encodings:
        # attempt to match each face in the input image to our known
        # encodings
        matches = face_recognition.compare_faces(data["encodings"], encoding, tolerance = tolerance)
        name = "unknown"

        # check to see if we have found a match
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}

            # loop over the matched indexes and maintain a count for
            # each recognized face face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1

            # determine the recognized face with the largest number of
            # votes (note: in the event of an unlikely tie Python will
            # select first entry in the dictionary)
            name = max(counts, key=counts.get)

        # update the list of names
        try:
            name = name.split('/')[2]
        except:
            name = name
        names.append(name)

    # loop over the recognized faces
    for ((top, right, bottom, left), name) in zip(boxes, names):
        # draw the predicted face name on the image
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)
    
    
    stop = time.time()
    #print('Time needed:', stop-start, 'sec')
    # show the output image
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if(show== True):
        print(names)
        plt.imshow(rgb)
        plt.show()
    return names

def imageRead(path, show=True):
    # BGR FORMAT #
    image = cv2.imread(path)
    
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if(show== True):
        plt.imshow(rgb)
        plt.show()
    return image
    comp
def encodingsRead(path):
    ## PAY ATTENTION:: You need to create the encoding for you dataset using encode_face.py
    data = pickle.loads(open(path, "rb").read())
    return data

In [51]:
def faceRecOnFolder(dataset_folder, encodings, correct_person, detection='cnn', min_threshold = 0.85, 
                    max_threshold=0.10, tolerance = 0.6, stopping_perc = 50, show=False):
    imagePaths = list(paths.list_images(dataset_folder))

    analysed_frames = 0
    correct_frames = 0
    negative_frames = 0
    neutrum_frames = 0
    total_frames = len(imagePaths)
    start = time.time()
    
    min_frames = total_frames * stopping_perc / 100
    # grab the paths to the input images in our dataset
    print("[INFO] Recognizing faces...")
    print("[INFO] elements:", total_frames)

    # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
        # extract the person name from the image path
        #print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
        image = imageRead(imagePath, show=False)
        person_rec = faceRecognition(image, encodings, detection='cnn', tolerance = tolerance, show=show)
        analysed_frames += 1
        
        if(len(person_rec) >= 1):
            person = person_rec[0]
            if(person == correct_person):
                correct_frames += 1
            else:
                if(person == 'unknown'):
                    neutrum_frames += 1
                else:
                    negative_frames += 1
        else:
            neutrum_frames += 1
        
            
        if( analysed_frames > min_frames and correct_frames/analysed_frames > min_threshold):
            print(correct_person, "Recognized after", analysed_frames, 'with accuracy:', correct_frames/analysed_frames)
            print('Not detected:', neutrum_frames/analysed_frames, ". Wrong recognition:", negative_frames/analysed_frames)
            correct_rate = correct_frames/analysed_frames
            neutrum_rate = neutrum_frames/analysed_frames
            negative_rate = negative_frames/analysed_frames
            end = time.time()
            print('Time:', end-start)
            return True, correct_rate, neutrum_rate, negative_rate
        
        if( analysed_frames > min_frames and correct_frames/analysed_frames < max_threshold):
            print(correct_person, "Recognized after", analysed_frames, 'with accuracy:', correct_frames/analysed_frames)
            print('Not detected:', neutrum_frames/analysed_frames, ". Wrong recognition:", negative_frames/analysed_frames)
            correct_rate = correct_frames/analysed_frames
            neutrum_rate = neutrum_frames/analysed_frames
            negative_rate = negative_frames/analysed_frames
            end = time.time()
            print('Time:', end-start)
            return False, correct_rate, neutrum_rate, negative_rate
        
    #for ends
    correct_rate = correct_frames/total_frames
    neutrum_rate = neutrum_frames/total_frames
    negative_rate = negative_frames/total_frames
    end = time.time()
    print(correct_person, "Recognized after", analysed_frames, 'with accuracy:', correct_frames/total_frames)
    print('Not detected:', neutrum_frames/total_frames, ". Wrong recognition:", negative_frames/total_frames)
    print('Time:', end-start)
    if(correct_rate > min_threshold - 0.05):
        return True, correct_rate, neutrum_rate, negative_rate
    else:
        return False, correct_rate, neutrum_rate, negative_rate

#### Encodings generation
Let's build new lighter encodings for each persons. 

For this reasons we modify the existing function to take a random and limited number of pictures for reach person.
However we won't use it.

In [52]:
def createEncondingsonFolder(folder, min_v = 0, max_v = 10000, enc_number = 20):
    start = time.time()
    for person in valid_persons:
        count = count +1
        if(count < min_v):
            continue
        if(count > max_v):
            break

        person = person[0:len(person)-1]
        test_folder = folder + person
        print(test_folder)
        # create the enconding related to first folder of the person
        folder_list = os.listdir(test_folder)
        first_n_folder = folder_list[0]
        train_folder = test_folder + "/" + first_n_folder
        res = os.path.exists("./encodings/" + person + ".pkl")
        if(res == False):
            createEncodings(train_folder, person, enc_number)
            
    end = time.time()
    print('Time needed', end-start)

In [53]:
#training_folder = '../#3/frame_images_DB/'
#createEncondingsonFolder(folder, min_v = 0, max_v = 1, enc_number = 20)

We can notice indeed that in the Aaron_Eckhart.plk training folder there are 85 pictures.
We mantain the same structure for the encoded files. So we can pick randomly a limited number of them from the already made ones.

In [54]:
def randomEncodingPicking(data, file_name, enc_number):
    new_enc = []
    new_names = []
    encodings = data['encodings']
    names = data['names']
    
    if( len(encodings) <= enc_number):
        new_enc = encodings
        new_names = names
    else:
        random_values = np.random.randint(0, len(encodings), size=enc_number)
        for r in random_values:
            new_enc.append(encodings[r])
            new_names.append(names[r])
            
    data = {"encodings": new_enc, "names": new_names}
    f = open('./encodings/' + file_name, "wb")
    f.write(pickle.dumps(data))
    f.close()
    return data

In [55]:
enc_folder = '../#3/encodings/'
name = 'Aaron_Eckhart.pkl'
data = encodingsRead(enc_folder + name)
print('Old number:', len(data['encodings']))
data = randomEncodingPicking(data, name, 20)
print('New number:', len(data['encodings']))

Old number: 85
New number: 20


In [56]:
def TrasformEncodings(source, enc_number):
    start = time.time()
    for file_name in os.listdir(source):
        print(file_name)
        data = encodingsRead(enc_folder + file_name)
        new_data = randomEncodingPicking(data, file_name, enc_number)

    end = time.time()
    print('Time needed', end-start)
    return

In [57]:
#TrasformEncodings(enc_folder, 20)

#### Tuning
Now we have light encodings. We can try to perform a faster training with multiple parameters. 

In [58]:
encodings_folder = "../#5/encodings/"
test_path = "../#3/frame_images_DB/"

In [59]:
### WE ARE DEFINING A FALSE THRESHOLD JUST TO MAKE THE TEST FASTER

def performTest(min_v=0, max_v=100000, true_threshold=0.85, false_threshold = 0.10, tolerance = 0.6, log=False):
    print("-------------------------")
    print("TEST STARTED")
    print("-------------------------")
    log_folder_base = 'test_tol' + '_' + str(tolerance) + '/'
    
    count = 0
    n_test = 0
    n_correct = 0
    filev = open("./valid.txt", "r")
    valid_persons = filev.readlines()
    filev.close()
    
    if( log == True):
        log_folder = log_folder_base + 'T/'
        try:
            os.mkdir("./" + log_folder_base)
            os.mkdir("./" + log_folder)
        except FileExistsError:
            print('Already existing folder')
        logfile = open("./" + log_folder + "log" + str(min_v) + "-" + str(max_v) + ".txt", "w")
                
    for person in valid_persons:
        count = count +1
        if(count < min_v):
            continue
        if(count > max_v):
            break

        person = person[0:len(person)-1]
        test_folder = test_path + person
        #test_folder = test_folder.replace("/", "\\")
        print(test_folder)
        # create the enconding related to first folder of the person
        folder_list = os.listdir(test_folder)
        first_n_folder = folder_list[0]
        train_folder = test_folder + "/" + first_n_folder
        res = os.path.exists("./encodings/" + person + ".pkl")
        if(res == False):
            createEncodings(train_folder, person)
        
        # test the encodings of testset
        train_flag = 0
        for n_folder in folder_list:
            if( train_flag == 0):
                train_flag = 1
                continue
            
            n_test += 1
            folder = test_folder + "/" + n_folder
            encoding = encodingsRead("./encodings/" + person + ".pkl")
            v, a, b, c = faceRecOnFolder("./" + folder, encoding, person, min_threshold= true_threshold,
                                             max_threshold = false_threshold,  tolerance = tolerance, show=False)
            if(log== True):
                line = person + " " + n_folder + " " + str(v) + " " + str(a) + " " + str(b) + " " + str(c) + "\n"
                logfile.write(line)

            if( v == True):
                n_correct += 1
    
    if(log == True):
        logfile.close()
    print("-------------------------")
    print("TEST ENDED")
    print("-------------------------")
    print('Accuracy on videos tested:', n_correct/n_test)
    return

Crossed Test needs lots of hours to be completely performed. 
We are defining a subset of the dataset on which performing the testing tasks (150 people over the total of 445 belonging to my part). 
We are then performing the cross test on a 10% percentage in every testing folder.

In [60]:
testing_limit =  150

def performCrossedTest(min_v=0, max_v=1000, true_threshold=0.85, false_threshold= 0.10, tolerance = 0.6, log=False):
    print("-------------------------")
    print("TEST STARTED")
    print("-------------------------")
    start = time.time()
    log_folder_base = 'test_tol' + '_' + str(tolerance) + '/'

    count = 0
    enc_list = os.listdir(encodings_folder)
    test_dir = os.listdir(test_path)
    
    filev = open("./valid.txt", "r")
    valid_persons = filev.readlines()
    filev.close()

    for encoding in enc_list:
        count = count + 1
        if(count < min_v):
            continue
        if(count > max_v):
            break
        n_test = 0
        n_correct = 0
        enc_name = encoding[0:len(encoding)-4]
        encoding_path = encodings_folder + encoding

        print('[ENC] Using encoding:', enc_name)

        
        if( log == True ):
            log_folder = log_folder_base + 'F/'
            try:
                #os.mkdir("./" + log_folder_base)
                os.mkdir("./" + log_folder)
            except FileExistsError:
                print('Already existing folder')
            logfile = open("./" + log_folder + "log_" + enc_name + ".txt", "w")

        p_count = 0
        for person in valid_persons:
            p_count += 1
            if( p_count > testing_limit):
                break
                
            person = person[0:len(person)-1]
            test_folder = test_path + person
            print('Cross testing on test folder:', person)
            folder_list = os.listdir(test_folder)

            person = person[0:len(person)]
            test_folder = test_path + person

            # test the encodings of testset
            train_flag = 0
            just_first = 0
            for n_folder in folder_list:
                if( enc_name == person ):
                    if( train_flag == 0):
                        train_flag = 1
                        just_first = 1
                        continue
                
                if( just_first == 1):
                    break
                    
                just_first +=1
                n_test += 1
                folder = test_folder + "/" + n_folder
                encoding = encodingsRead(encoding_path)
                #imagePaths = list(os.listdir(test_path + person + "/"))

                print(folder)
                v, a, b, c = faceRecOnFolder("./" + folder, encoding, person, min_threshold=true_threshold,
                                             max_threshold = false_threshold, tolerance = tolerance, 
                                             stopping_perc = 20, show=False)
                if(log== True):
                    line = person + " " + n_folder + " " + str(v) + " " + str(a) + " " + str(b) + " " + str(c) + "\n"
                    logfile.write(line)

                if( v == True):
                    n_correct += 1

        if(log == True):
            logfile.close()
    
    end = time.time()
    print("-------------------------")
    print("TEST ENDED")
    print("-------------------------")
    print('This test last:', end-start)
    return

#### Running test
We are defining a range of tolerances. We will analyse the thresholds after, thanks to the logs' file.

In [ ]:
tol_values = [ 0.6, 0.5, 0.4, 0.3, 0.2, 0.1 ]


for tol in tol_values:
    print('**********************')
    print('TOLERANCE', tol)
    print('**********************')
    performTest(min_v=0, max_v=150, true_threshold = 0.70, tolerance = tol, log=True)
    performCrossedTest(min_v=0, max_v=5, true_threshold = 0.70, tolerance = tol, log=True)

**********************
TOLERANCE 0.6
**********************
-------------------------
TEST STARTED
-------------------------
Already existing folder
../#3/frame_images_DB/Aaron_Eckhart
[INFO] Recognizing faces...
[INFO] elements: 49
Aaron_Eckhart Recognized after 25 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 4.149955987930298
../#3/frame_images_DB/Aaron_Tippin
[INFO] Recognizing faces...
[INFO] elements: 83
Aaron_Tippin Recognized after 42 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 12.17406702041626
../#3/frame_images_DB/Abba_Eban
[INFO] Recognizing faces...
[INFO] elements: 237
Abba_Eban Recognized after 119 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 20.353777408599854
../#3/frame_images_DB/Abdel_Aziz_Al-Hakim
[INFO] Recognizing faces...
[INFO] elements: 285
Abdel_Aziz_Al-Hakim Recognized after 285 with accuracy: 0.12280701754385964
Not detected: 0.8771929824561403 . Wrong recognition: 0.0
Time: 68.138025760650

Alec_Baldwin Recognized after 63 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 16.892312049865723
../#3/frame_images_DB/Alejandro_Gonzalez_Inarritu
[INFO] Recognizing faces...
[INFO] elements: 100
Alejandro_Gonzalez_Inarritu Recognized after 51 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 13.30387282371521
[INFO] Recognizing faces...
[INFO] elements: 61
Alejandro_Gonzalez_Inarritu Recognized after 31 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 13.861576080322266
../#3/frame_images_DB/Alejandro_Toledo
[INFO] Recognizing faces...
[INFO] elements: 154
Alejandro_Toledo Recognized after 78 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 12.749218940734863
[INFO] Recognizing faces...
[INFO] elements: 119
Alejandro_Toledo Recognized after 60 with accuracy: 0.75
Not detected: 0.25 . Wrong recognition: 0.0
Time: 9.879764795303345
../#3/frame_images_DB/Aleksander_Kwasniewski
[INFO] Recognizing faces...
[INFO

Alison_Lohman Recognized after 59 with accuracy: 0.2542372881355932
Not detected: 0.7457627118644068 . Wrong recognition: 0.0
Time: 14.411271333694458
[INFO] Recognizing faces...
[INFO] elements: 153
Alison_Lohman Recognized after 77 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 21.73936104774475
[INFO] Recognizing faces...
[INFO] elements: 74
Alison_Lohman Recognized after 38 with accuracy: 0.7368421052631579
Not detected: 0.2631578947368421 . Wrong recognition: 0.0
Time: 17.103546619415283
../#3/frame_images_DB/Ali_Abbas
[INFO] Recognizing faces...
[INFO] elements: 65
Ali_Abbas Recognized after 33 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 14.841758012771606
[INFO] Recognizing faces...
[INFO] elements: 61
Ali_Abbas Recognized after 31 with accuracy: 0.8709677419354839
Not detected: 0.12903225806451613 . Wrong recognition: 0.0
Time: 5.291871070861816
[INFO] Recognizing faces...
[INFO] elements: 105
Ali_Abbas Recognized after 53 with accur

Amy_Redford Recognized after 31 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 7.030223608016968
../#3/frame_images_DB/Amy_Yasbeck
[INFO] Recognizing faces...
[INFO] elements: 107
Amy_Yasbeck Recognized after 54 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 7.786953449249268
[INFO] Recognizing faces...
[INFO] elements: 85
Amy_Yasbeck Recognized after 43 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 11.305514335632324
../#3/frame_images_DB/Andrea_Bocelli
[INFO] Recognizing faces...
[INFO] elements: 85
Andrea_Bocelli Recognized after 43 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 8.05691385269165
[INFO] Recognizing faces...
[INFO] elements: 53
Andrea_Bocelli Recognized after 27 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 12.080625534057617
../#3/frame_images_DB/Andrea_De_Cruz
[INFO] Recognizing faces...
[INFO] elements: 97
Andrea_De_Cruz Recognized after 49 with accuracy: 0.0


Barbara_Brezigar Recognized after 46 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 12.964922189712524
[INFO] Recognizing faces...
[INFO] elements: 69
Barbara_Brezigar Recognized after 35 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 9.826079368591309
../#3/frame_images_DB/Barbara_Walters
[INFO] Recognizing faces...
[INFO] elements: 93
Barbara_Walters Recognized after 47 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 8.609566926956177
[INFO] Recognizing faces...
[INFO] elements: 61
Barbara_Walters Recognized after 31 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 6.2898170948028564
[INFO] Recognizing faces...
[INFO] elements: 277
Barbara_Walters Recognized after 139 with accuracy: 1.0
Not detected: 0.0 . Wrong recognition: 0.0
Time: 25.808643341064453
../#3/frame_images_DB/Barbra_Streisand
[INFO] Recognizing faces...
[INFO] elements: 68
Barbra_Streisand Recognized after 68 with accuracy: 0.411764705882

...

Alex_Ferguson Recognized after 10 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 2.245993137359619
Cross testing on test folder: Alex_Holmes
../#3/frame_images_DB/Alex_Holmes/4
[INFO] Recognizing faces...
[INFO] elements: 50
Alex_Holmes Recognized after 11 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 1.8959312438964844
Cross testing on test folder: Alex_Sink
../#3/frame_images_DB/Alex_Sink/4
[INFO] Recognizing faces...
[INFO] elements: 598
Alex_Sink Recognized after 120 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 33.31765079498291
Cross testing on test folder: Alfonso_Cuaron
../#3/frame_images_DB/Alfonso_Cuaron/0
[INFO] Recognizing faces...
[INFO] elements: 650
Alfonso_Cuaron Recognized after 131 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 36.34884190559387
Cross testing on test folder: Alfredo_di_Stefano
../#3/frame_images_DB/Alfredo_di_Stefano/0
[INFO] Recognizing faces...
[INFO] elements: 55


Andres_Manuel_Lopez_Obrador Recognized after 19 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 2.9805822372436523
Cross testing on test folder: Andrew_Bernard
../#3/frame_images_DB/Andrew_Bernard/0
[INFO] Recognizing faces...
[INFO] elements: 141
Andrew_Bernard Recognized after 29 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 6.498625755310059
Cross testing on test folder: Andrew_Cuomo
../#3/frame_images_DB/Andrew_Cuomo/2
[INFO] Recognizing faces...
[INFO] elements: 69
Andrew_Cuomo Recognized after 14 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 3.144552230834961
Cross testing on test folder: Andrew_Wetzler
../#3/frame_images_DB/Andrew_Wetzler/0
[INFO] Recognizing faces...
[INFO] elements: 221
Andrew_Wetzler Recognized after 45 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 7.060466766357422
Cross testing on test folder: Andre_Agassi
../#3/frame_images_DB/Andre_Agassi/4
[INFO] Recognizing faces...
[I

Benjamin_McKenzie Recognized after 19 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 3.2642319202423096
Cross testing on test folder: Benjamin_Netanyahu
../#3/frame_images_DB/Benjamin_Netanyahu/0
[INFO] Recognizing faces...
[INFO] elements: 89
Benjamin_Netanyahu Recognized after 18 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 5.54164981842041
Cross testing on test folder: Ben_Broussard
../#3/frame_images_DB/Ben_Broussard/2
[INFO] Recognizing faces...
[INFO] elements: 61
Ben_Broussard Recognized after 13 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 3.763460159301758
Cross testing on test folder: Ben_Curtis
../#3/frame_images_DB/Ben_Curtis/0
[INFO] Recognizing faces...
[INFO] elements: 169
Ben_Curtis Recognized after 34 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 9.447732210159302
Cross testing on test folder: Ben_Kingsley
../#3/frame_images_DB/Ben_Kingsley/1
[INFO] Recognizing faces...
[INFO] elem

Carlos_Iturgaitz Recognized after 26 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 4.053190469741821
Cross testing on test folder: Carlos_Menem
../#3/frame_images_DB/Carlos_Menem/0
[INFO] Recognizing faces...
[INFO] elements: 48
Carlos_Menem Recognized after 10 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 0.9868948459625244
Cross testing on test folder: Carlos_Queiroz
../#3/frame_images_DB/Carlos_Queiroz/0
[INFO] Recognizing faces...
[INFO] elements: 56
Carlos_Queiroz Recognized after 12 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 1.8667588233947754
Cross testing on test folder: Carlos_Quintanilla_Schmidt
../#3/frame_images_DB/Carlos_Quintanilla_Schmidt/1
[INFO] Recognizing faces...
[INFO] elements: 534
Carlos_Quintanilla_Schmidt Recognized after 107 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 29.57019805908203
Cross testing on test folder: Carlos_Salinas
../#3/frame_images_DB/Carlos_Salinas/0


Alberto_Acosta Recognized after 46 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 7.217733860015869
Cross testing on test folder: Alberto_Gonzales
../#3/frame_images_DB/Alberto_Gonzales/2
[INFO] Recognizing faces...
[INFO] elements: 84
Alberto_Gonzales Recognized after 17 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 3.1598758697509766
Cross testing on test folder: Alberto_Ruiz_Gallardon
../#3/frame_images_DB/Alberto_Ruiz_Gallardon/0
[INFO] Recognizing faces...
[INFO] elements: 197
Alberto_Ruiz_Gallardon Recognized after 40 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 10.078573942184448
Cross testing on test folder: Albert_Brooks
../#3/frame_images_DB/Albert_Brooks/0
[INFO] Recognizing faces...
[INFO] elements: 217
Albert_Brooks Recognized after 44 with accuracy: 0.0
Not detected: 1.0 . Wrong recognition: 0.0
Time: 6.880658149719238
Cross testing on test folder: Alec_Baldwin
../#3/frame_images_DB/Alec_Baldwin/0
[INFO] Re

#### Statistichs
Let's show a ROC curve for every

In [ ]:
import os
import numpy as np

values = [0.0, 0.45, 0.50, 0.55, 0.60, 0.65, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99, 0.9999]
confirmed_rate = np.zeros(len(values))

def statsOnCrossedTest(tol):
    print('Tolerance', tol)
    log_folder = "./test_tol_" + str(tol) + "/F/"
    print(log_folder)
    files = os.listdir(log_folder)
    i = 0

    for thr in thr_values:
        total_files = 0
        recognized = 0
        notdetected = 0
        wrong = 0
        wrong_confirmed = 0
        wrong_confirmed2 = 0
        wrong_confirmed3 = 0
        wrong_confirmed4 = 0

        ####
        one_wrong_rec = 0
        persons = 0
        for f in files:
            if( f.startswith('log') == True):
                filev = open(log_folder + f, "r")
                vector = f.split('_')
                correct_person = vector[1] + "_" + vector[2]
                lines = filev.readlines()
                persons += 1
                for line in lines: 
                    #we don't want to make statistics on the same person
                    if(line.split(' ')[0] == correct_person):
                        continue

                    total_files += 1

                    rec_flag = line.split(' ')[2]
                    rec_percentage = float(line.split(' ')[3])
                    notdec_percentage = float(line.split(' ')[4])
                    notrec_percentage = float(line.split(' ')[5])

                    recognized = recognized + rec_percentage
                    notdetected = notdetected + notdec_percentage
                    wrong = wrong + notrec_percentage

                    if(notrec_percentage > thr):
                        wrong_confirmed = wrong_confirmed + 1

        confirmed_rate[i] = wrong_confirmed / total_files
        wrong_confirmed = 0
        i = i +1

    ## equal for each iteration
    positive = recognized / total_files 
    undetected = notdetected / total_files
    negative = wrong / total_files

    wrong_rec_on_persons = one_wrong_rec / (persons-1)
    return positive, undetected, negative, confirmed_rate

In [ ]:
## WRONG ONES WHERE ASSIGNED TO THE PERSON EVEN HE WASN'T
tol_values = [ 0.6, 0.5, 0.4, 0.3, 0.2, 0.1 ]

for tol in tol_values:
    positive, undetected, negative, far = statsOnCrossedTest(tol)
    labels = ['Correct', 'NotRecognized', 'Wrong']
    colors = ['Blue', 'Yellow', 'Orange']
    sizes1 = [positive, undetected, negative]
    print('Percentage of overall images that were correctly not recognized:', undetected)

    plt.pie(sizes1, labels=labels, shadow=True, startangle=90, colors=colors)
    plt.title('Recognition average over all the images')
    plt.legend()
    plt.show()

In [ ]:
import numpy as np
import os


def statsOnTest(tol):
    TAR_path = "./test_tol_" + str(tol) + "/T/"
    files = os.listdir(TAR_path)
    confirmed_rateT = np.zeros(len(values))
    print('Tolerance', tol)
    print(TAR_path)
    files = os.listdir(TAR_path)
    i = 0
    i=0

    for thr in values:
        total_files = 0
        recognized = 0
        notdetected = 0
        wrong = 0
        confirmed = 0
        total_files = 0

        for f in files:
            if( f.startswith('log') == True):
                filev = open(TAR_path + f, "r")
                lines = filev.readlines()
                for line in lines:
                    total_files += 1

                    rec_flag = line.split(' ')[2]
                    rec_percentage = float(line.split(' ')[3])
                    notdec_percentage = float(line.split(' ')[4])
                    notrec_percentage = float(line.split(' ')[5])

                    recognized = recognized + rec_percentage
                    notdetected = notdetected + notdec_percentage
                    wrong = wrong + notrec_percentage
                    if(rec_percentage > thr):
                        confirmed = confirmed + 1

        confirmed_rateT[i] = confirmed / total_files

        positive = recognized / total_files 
        undetected = notdetected / total_files
        negative = wrong / total_files

        i = i + 1
    
    return positive, undetected, negative, confirmed_rateT

In [ ]:
from sklearn.metrics import roc_curve, auc
for tol in tol_values:
    _, _, _, far = statsOnCrossedTest(tol)
    _, _, _, tar = statsOnTest(tol)
    fpr = far
    tpr = tar
    exampleX = [0.0, 0.001, 0.2, 0.8, 1.0]
    exampleY = [0.0, 1.0, 1.0, 1.0, 1.0]

    plt.figure()
    lw = 2
    plt.plot(exampleX, exampleY, color='green', linestyle='-', label='Ideal ROC')
    plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='Random case')
    plt.xlim([-0.01, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

    plt.figure()
    lw = 2
    plt.plot(exampleX, exampleY, color='green', linestyle='-', label='Ideal ROC')
    plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='Random case')
    plt.xlim([0.0, 0.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Zooming in the ROC curve')
    plt.legend(loc="lower right")
    plt.show()